In [6]:
pip install geopandas shapely

Note: you may need to restart the kernel to use updated packages.


In [62]:
import geopandas as gpd
from shapely.geometry import LineString, Point

In [99]:
# Load shapefiles with forward slashes
bankline_shp = gpd.read_file("D:/BUET/Thesis/ArcGIS/Bankline/2024.shp")  
waterbody_shp = gpd.read_file("D:/BUET/Thesis/ArcGIS/River TIF/Water_SHP/2024.shp")  
cross_sections_shp = gpd.read_file("D:/BUET/Thesis/ArcGIS/Cross_Section/Cross_section.shp")

In [100]:
# Ensure all data are in the same coordinate reference system (CRS)
target_crs = bankline_shp.crs
waterbody_shp = waterbody_shp.to_crs(target_crs)
cross_sections_shp = cross_sections_shp.to_crs(target_crs)

In [101]:
# Compute distance between banklines along each cross-section
distances = []
for index, cross_section in cross_sections_shp.iterrows():
    # Buffer the cross-section to create an intersection area with the banklines
    cross_line = cross_section.geometry
    bank_intersections = bankline_shp.intersection(cross_line)
    waterbody_intersections = waterbody_shp.intersection(cross_line)

    # Calculate distance between two bankline points on the cross-section
    bank_points = [p for p in bank_intersections if not p.is_empty]
    if len(bank_points) >= 2:
        bank_distance = bank_points[0].distance(bank_points[1])
    else:
        bank_distance = None  # Not enough intersection points for measurement
    
    # Sum the lengths of waterbody segments that are at least 80 meters
    qualifying_water_lengths = sum(w.length for w in waterbody_intersections if not w.is_empty and w.length >= 80)

    # Convert distances from meters to kilometers
    if bank_distance is not None:
        bank_distance /= 1000  # Convert bankline distance to km
    qualifying_water_lengths /= 1000  # Convert waterbody length to km

    # Store results for each cross-section
    distances.append({
        "cross_section_id": cross_section["Id"],  # Assuming each cross-section has an ID field
        "bankline_distance": bank_distance,
        "waterbody_length": qualifying_water_lengths
    })

In [102]:
# Convert results to a GeoDataFrame for easier analysis and exporting
distances_gdf = gpd.GeoDataFrame(distances)

# Optional: Save results to a new shapefile or CSV
distances_gdf.to_csv("cross_section_distances_2024.csv", index=False)  # To save as a CSV file

# Print the results
print(distances_gdf)

    cross_section_id  bankline_distance  waterbody_length
0                  1          10.955135          4.364492
1                  2           9.516375          2.192629
2                  3          12.306818          3.288727
3                  4          14.714884          2.920279
4                  5          14.400389          3.037890
5                  6          15.067635          2.680038
6                  7          13.637407          2.972021
7                  8          12.298553          2.627590
8                  9           9.654935          2.787641
9                 10          10.373041          2.420730
10                11          13.545766          2.476261
11                12          14.405426          2.387510
12                13          14.901520          2.502257
13                14          12.100940          2.748764
14                15          15.034970          2.731725
15                16          13.442071          2.582636
16            